# 3-1. Cleaning
> Garbage In, Garbage Out

- 클리닝, 필터링 -> 특징 선택 (피처 엔지니어링)  
- [Amazing Feature Engineering](https://github.com/ashishpatel26/Amazing-Feature-Engineering)
- 결측값, 오염된 값으로 인해 잘못된 인사이트를 도출할 수 있음

## 결측치